# My titanic contest

In [461]:
import numpy as np
import pandas as pd
from sklearn import model_selection, preprocessing, linear_model, model_selection, metrics, ensemble
from sklearn.ensemble import RandomForestClassifier


In [462]:
data = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
print(data.shape)
print(data.head())

(891, 12)
   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN    

In [463]:
print(test.shape)
print(test.head())

(418, 11)
   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


In [464]:
test_ids = test.iloc[:,0]

## 1. Пропущенные значения

In [465]:
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [466]:
#Разберемся с пропущенными значениями возраста
mean_age = round(data['Age'].mean(),1)
median_age = round(data['Age'].median(),1)
print(mean_age)
print(median_age)

data['Age'].fillna(median_age, inplace = True)

29.7
28.0


In [467]:
#Разберемся с пропущенными значениями порта отправления
moda_embarked = data['Embarked'].mode()[0]
data.Embarked.fillna(moda_embarked, inplace = True)
data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64

In [468]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [469]:
#пропущенные значения в test
mean_fare = round(data['Fare'].mean(),1)
test['Age'].fillna(median_age, inplace = True)
test['Fare'].fillna(mean_fare, inplace = True)
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [470]:
#data[data.Cabin.isnull()==False]

## 2. Кодирование категориальных признаков

In [471]:
data['Sex'] = preprocessing.LabelEncoder().fit_transform(data['Sex'])
test['Sex'] = preprocessing.LabelEncoder().fit_transform(test['Sex'])
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [472]:
data['Embarked'] = data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype('int')
test['Embarked'] = test['Embarked'].map({'S': 0, 'C': 1, 'Q': 2}).astype('int')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,0
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,0


## 3. Удаляем неиспользуемые колонки

In [473]:
data.drop(columns=['Ticket'], inplace=True)
test.drop(columns=['Ticket'], inplace=True)

data.drop(columns=['Cabin'], inplace=True)
test.drop(columns=['Cabin'], inplace=True)

data.drop(columns=['Name'], inplace=True)
test.drop(columns=['Name'], inplace=True)

data.drop(columns=['PassengerId'], inplace=True)
test.drop(columns=['PassengerId'], inplace=True)

In [474]:
data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,0
1,1,1,0,38.0,1,0,71.2833,1
2,1,3,0,26.0,0,0,7.9250,0
3,1,1,0,35.0,1,0,53.1000,0
4,0,3,1,35.0,0,0,8.0500,0


## 4. Обучение модели

In [475]:
y = data['Survived']
X = data.drop(columns=['Survived'])
(X_train, X_test, y_train, y_test) = model_selection.train_test_split(X, y, test_size=0.3, random_state=0)

In [476]:
X_test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
495,3,1,28.0,0,0,14.4583,1
648,3,1,28.0,0,0,7.5500,0
278,3,1,7.0,4,1,29.1250,2
31,1,0,28.0,1,0,146.5208,1
255,3,0,29.0,0,2,15.2458,1


### Логистическая регрессия

In [477]:
model = linear_model.LogisticRegression(random_state=0, solver='lbfgs')
model.fit(X_train, y_train)
result = model.predict(X_test)
roc_auc = metrics.roc_auc_score(y_test, result)
print("ROC AUC: ", roc_auc)
accuracy = metrics.accuracy_score(y_test, result)
print("Accuracy: ", accuracy)

ROC AUC:  0.7976190476190477
Accuracy:  0.8097014925373134


D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


### Random forest

In [478]:
rf_model = ensemble.RandomForestClassifier()
rf_model.fit(X_train, y_train)
rf_result = rf_model.predict(X_test)
rf_roc_auc = metrics.roc_auc_score(y_test, rf_result)
print("ROC AUC: ", rf_roc_auc)
rf_accuracy = metrics.accuracy_score(y_test, rf_result)
print("Accuracy: ", rf_accuracy)

ROC AUC:  0.8076190476190476
Accuracy:  0.8171641791044776


D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [479]:
result = rf_model.predict(test)
submission = pd.DataFrame({'PassengerId':test_ids,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)
filename = 'Titanic Predictions_1.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

(418, 2)
Saved file: Titanic Predictions_1.csv


## 5. Доработка набора данных

In [480]:
data['Relatives'] = data['SibSp'] + data['Parch']
test['Relatives'] = test['SibSp'] + test['Parch']

data['Alone'] = 0
data.loc[data['Relatives'] == 0, 'Alone'] = 1
test['Alone'] = 0
test.loc[test['Relatives'] == 0, 'Alone'] = 1
test.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Relatives,Alone
0,3,1,34.5,0,0,7.8292,2,0,1
1,3,0,47.0,1,0,7.0000,0,1,0
2,2,1,62.0,0,0,9.6875,2,0,1
3,3,1,27.0,0,0,8.6625,0,0,1
4,3,0,22.0,1,1,12.2875,0,2,0


In [481]:
data.drop(columns=['SibSp', 'Parch'], inplace=True)
test.drop(columns=['SibSp', 'Parch'], inplace=True)

## 6. Новая итерация обучения

In [482]:
y_new = data['Survived']
X_new = data.drop(columns=['Survived'])
(X_train_new, X_test_new, y_train_new, y_test_new) = model_selection.train_test_split(X_new, y_new, test_size=0.3, random_state=0)

In [483]:
new_model = linear_model.LogisticRegression(random_state=0, solver='lbfgs')
new_model.fit(X_train_new, y_train_new)
new_result = new_model.predict(X_test_new)
new_roc_auc = metrics.roc_auc_score(y_test_new, new_result)
print("ROC AUC: ", new_roc_auc)
new_accuracy = metrics.accuracy_score(y_test_new, new_result)
print("Accuracy: ", new_accuracy)

ROC AUC:  0.7826190476190477
Accuracy:  0.7985074626865671


D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [484]:
new_rf_model = ensemble.RandomForestClassifier()
new_rf_model.fit(X_train_new, y_train_new)
new_rf_result = new_rf_model.predict(X_test_new)
new_rf_roc_auc = metrics.roc_auc_score(y_test_new, new_rf_result)
print("ROC AUC: ", new_rf_roc_auc)
new_rf_accuracy = metrics.accuracy_score(y_test_new, new_rf_result)
print("Accuracy: ", new_rf_accuracy)

ROC AUC:  0.7834523809523809
Accuracy:  0.8097014925373134


D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [485]:
result = new_rf_model.predict(test)
submission = pd.DataFrame({'PassengerId':test_ids,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)
filename = 'Titanic Predictions_20.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

(418, 2)
Saved file: Titanic Predictions_20.csv


## 7. Еще одна итерация

In [486]:
data['Fare_cat']=0
test['Fare_cat']=0
median_fare = data['Fare'].median()
data.loc[data['Fare'] > median_fare, 'Fare_cat'] = 1
test.loc[test['Fare'] > median_fare, 'Fare_cat'] = 1

data.drop(columns=['Fare'], inplace=True)
test.drop(columns=['Fare'], inplace=True)

data.head()

,Survived,Pclass,Sex,Age,Embarked,Relatives,Alone,Fare_cat
0,0,3,1,22.0,0,1,0,0
1,1,1,0,38.0,1,1,0,1
2,1,3,0,26.0,0,0,1,0
3,1,1,0,35.0,0,1,0,1
4,0,3,1,35.0,0,0,1,0


In [487]:
y_1 = data['Survived']
X_1 = data.drop(columns=['Survived'])
(X_train_1, X_test_1, y_train_1, y_test_1) = model_selection.train_test_split(X_1, y_1, test_size=0.3, random_state=0)

In [488]:
_model = linear_model.LogisticRegression(random_state=0, solver='lbfgs')
_model.fit(X_train_1, y_train_1)
_result = _model.predict(X_test_1)
_roc_auc = metrics.roc_auc_score(y_test_1, _result)
print("ROC AUC: ", _roc_auc)
_accuracy = metrics.accuracy_score(y_test_1, _result)
print("Accuracy: ", _accuracy)

ROC AUC:  0.7876190476190477
Accuracy:  0.8022388059701493


In [489]:
_rf_model = ensemble.RandomForestClassifier()
_rf_model.fit(X_train_1, y_train_1)
_rf_result = _rf_model.predict(X_test_1)
_rf_roc_auc = metrics.roc_auc_score(y_test_1, _rf_result)
print("ROC AUC: ", _rf_roc_auc)
_rf_accuracy = metrics.accuracy_score(y_test_1, _rf_result)
print("Accuracy: ", _rf_accuracy)

ROC AUC:  0.7885714285714286
Accuracy:  0.8059701492537313


D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [380]:
result = _rf_model.predict(test)
submission = pd.DataFrame({'PassengerId':test_ids,'Survived':result})
submission.Survived = submission.Survived.astype(int)
print(submission.shape)
filename = 'Titanic Predictions_2.csv'
submission.to_csv(filename,index=False)
print('Saved file: ' + filename)

(418, 2)
Saved file: Titanic Predictions_2.csv
